# 🗺️ Flat Earth Experiment

### Originally created by Torsten Auerswald (NCAS, University of Reading)

## 🎯 Objective

In this experiment, you will be using SpeedyWeather to
investigate the influence of continents, and especially large mountains, on the global circulation of
the atmosphere. You will use this model to investigate the role of mountains for the general
circulation and climate on Earth.

## 🗻 🧪 Flat Earth Experiment Setup 
This notebook is for running SpeedyWeather with and without orography. 

**🔹 To run the flat earth run** define a new orography without mountains and add it to the model definition:

`orography=NoOrography(spectral_grid)`

`model = PrimitiveWetModel(spectral_grid; orography,vertical_advection, albedo, output)`

**🔹 To run the control run** remove the orography input from the definition of the model. This will activate the default earth orography when the simulation is initialized.

`model = PrimitiveWetModel(spectral_grid; vertical_advection, albedo, output)`

👉 Conduct both simulations (control run and flat earth experiment) and remember to change the run id in each simulation. After the simulations have finished you can use the notebook "Flat_Earth_analysis.ipynb" to have a first look at the data and get some ideas for further analysis.

## 🔬 🧠 Guiding questions
 - How does the midlatitude circulation change in the flat earth experiment? Have a look at surface pressure, and geopotential height and winds at different levels.
   
 - What is the role of topography over East Africa and the Tibetan Plateau?

 - Do the meridional atmospheric transports of heat and moisture change?

## 💪📚 Further Reading
For more details on the theoretical background and literature have a look at the lab notes for this experiment in "Flat_Earth_Labnotes.pdf"

In [1]:
using Pkg

In [ ]:
Pkg.status("SpeedyWeather")

In [ ]:
#Use this only to update to a new version of SpeedyWeather
#Main release
#Pkg.add("SpeedyWeather")
#Developer version
#Pkg.develop("SpeedyWeather")

In [ ]:
using SpeedyWeather
using CairoMakie

# Define grid
spectral_grid = SpectralGrid(trunc=31, nlayers=8)
# Define output. Change the id for the control run and the flat earth run
output = NetCDFOutput(spectral_grid, PrimitiveWetModel, path="./", id="Flat_Earth", output_dt=Minute(360))



In [ ]:
# Some parameterisation choices
vertical_advection = WENOVerticalAdvection(spectral_grid)

#Define albedo
albedo = Albedo(ocean=OceanSeaIceAlbedo(spectral_grid), land=AlbedoClimatology(spectral_grid))


In [ ]:
### Set up the model to run from scratch
#Run this for the simulation without orography
orography=NoOrography(spectral_grid)
model = PrimitiveWetModel(spectral_grid; orography,vertical_advection, albedo, output)
#Run this for the control simulation with original orography
#model = PrimitiveWetModel(spectral_grid; vertical_advection, albedo, output)

In [ ]:
simulation = initialize!(model)
# now offset via set! with add=true
set!(albedo.land, (λ, φ) -> 0.1, add=true)

# Add output variables
add!(model, SpeedyWeather.OceanOutput()...)
add!(model, SpeedyWeather.LandOutput()...)
add!(model, SpeedyWeather.RadiationOutput()...)
add!(model, SpeedyWeather.PrecipitationOutput()...)
add!(model, SpeedyWeather.SurfaceFluxesOutput()...)
#This adds orography variable to output
add!(model, SpeedyWeather.BoundaryOutput()...)



In [ ]:
#Check model settings
model

In [ ]:
#Plot orography to make sure we set it to zero
heatmap(model.orography.orography)

In [ ]:
#Run the simulation
run!(simulation, period=Year(30), output=true)